#### Análise de Indicadores de Performance em Rede de Venda

In [248]:
# Versão da linguagem Python
from platform import python_version
print('Versão da linguam Python usada nesse código:', python_version())

Versão da linguam Python usada nesse código: 3.9.2


In [249]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import plotly
import matplotlib.pyplot as plt
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
from datetime import datetime, timedelta
%matplotlib inline
#pyoff.init_notebook_mode()

In [250]:
# Versões dos pacotes utilizados nesse código

%reload_ext watermark
%watermark -a "Maria Eduarda" --iversions

Author: Maria Eduarda

numpy     : 1.20.2
pandas    : 1.2.3
plotly    : 4.2.0
seaborn   : 0.11.2
matplotlib: 3.4.2



In [251]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy     : 1.20.2
pandas    : 1.2.3
plotly    : 4.2.0
seaborn   : 0.11.2
matplotlib: 3.4.2



#### Carregamento e tratamento dos dados

In [253]:
# fonte: https://www.kaggle.com/datasets/heemalichaudhari/adidas-sales-dataset 
dados = pd.read_excel('Adidas US Sales Datasets.xlsx')
#dados = dados.iloc[3::,]  # exclui as 3 primeiras linhas em branco
dados.drop(dados.columns[[0]],axis = 1, inplace=True) # exclui primeira coluna em branco
dados.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,Adidas Sales Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
4,Foot Locker,1185732,2020-01-01 00:00:00,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000,0.5,In-store


In [254]:
# Renomeia cabeçalho com linha selecionada
cabecalho = list(dados.iloc[3,:])
dados.columns = cabecalho
# Exclui as 3 primeiras linhas em branco
dados = dados.iloc[4::,:] 
dados = dados.reset_index(drop=True)
dados.head()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,2020-01-01 00:00:00,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000,0.5,In-store
1,Foot Locker,1185732,2020-01-02 00:00:00,Northeast,New York,New York,Men's Athletic Footwear,50,1000,500000,150000,0.3,In-store
2,Foot Locker,1185732,2020-01-03 00:00:00,Northeast,New York,New York,Women's Street Footwear,40,1000,400000,140000,0.35,In-store
3,Foot Locker,1185732,2020-01-04 00:00:00,Northeast,New York,New York,Women's Athletic Footwear,45,850,382500,133875,0.35,In-store
4,Foot Locker,1185732,2020-01-05 00:00:00,Northeast,New York,New York,Men's Apparel,60,900,540000,162000,0.3,In-store


In [255]:
# Renomeia as colunas para o português
dados.rename(columns={
    'Retailer' : 'Revendedor',
    'Retailer ID' : 'ID Revendedor',
    'Invoice Date' : 'Data da Fatura',
    'Region' : 'Regiao',
    'State' : 'Estado',
    'City' : 'Cidade',
    'Product' : 'Produto',
    'Price per Unit' : 'Preco por Unidade',
    'Units Sold' : 'Unidades Vendidas',
    'Total Sales' : 'Total Venda',
    'Operating Profit' : 'Lucro Operacional',
    'Operating Margin' : 'Margem de Lucro',
    'Sales Method' : 'Metodo de Venda'
}, inplace=True)

In [256]:
dados.isna().sum()

Revendedor           0
ID Revendedor        0
Data da Fatura       0
Regiao               0
Estado               0
Cidade               0
Produto              0
Preco por Unidade    0
Unidades Vendidas    0
Total Venda          0
Lucro Operacional    0
Margem de Lucro      0
Metodo de Venda      0
dtype: int64

In [257]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Revendedor         9648 non-null   object
 1   ID Revendedor      9648 non-null   object
 2   Data da Fatura     9648 non-null   object
 3   Regiao             9648 non-null   object
 4   Estado             9648 non-null   object
 5   Cidade             9648 non-null   object
 6   Produto            9648 non-null   object
 7   Preco por Unidade  9648 non-null   object
 8   Unidades Vendidas  9648 non-null   object
 9   Total Venda        9648 non-null   object
 10  Lucro Operacional  9648 non-null   object
 11  Margem de Lucro    9648 non-null   object
 12  Metodo de Venda    9648 non-null   object
dtypes: object(13)
memory usage: 980.0+ KB


Modifca tipo dos dados

In [259]:
dados['Unidades Vendidas'] = dados['Unidades Vendidas'].astype(int)

In [260]:
dados[['Margem de Lucro','Preco por Unidade','Total Venda', 'Lucro Operacional']] = dados[['Margem de Lucro','Preco por Unidade','Total Venda', 'Lucro Operacional']].astype(float)

In [261]:
dados['Data da Fatura'] = pd.to_datetime(dados['Data da Fatura'], format="%Y-%m-%d")

In [263]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Revendedor         9648 non-null   object        
 1   ID Revendedor      9648 non-null   object        
 2   Data da Fatura     9648 non-null   datetime64[ns]
 3   Regiao             9648 non-null   object        
 4   Estado             9648 non-null   object        
 5   Cidade             9648 non-null   object        
 6   Produto            9648 non-null   object        
 7   Preco por Unidade  9648 non-null   float64       
 8   Unidades Vendidas  9648 non-null   int32         
 9   Total Venda        9648 non-null   float64       
 10  Lucro Operacional  9648 non-null   float64       
 11  Margem de Lucro    9648 non-null   float64       
 12  Metodo de Venda    9648 non-null   object        
dtypes: datetime64[ns](1), float64(4), int32(1), object(7)
memory us

##### Criação de coluna AnoMes

In [264]:
# Extrai ano da coluna
# Define a função para aplicar a cada registro da coluna
def extrair_dia(data):
    return data.strftime('%y')

# Adiciona a nova coluna com os anos extraídos da coluna de data
dados['Ano'] = dados['Data da Fatura'].apply(extrair_dia)

In [265]:
# Formata ano
# Define a função para aplicar a cada registro da coluna
def formata_ano(data):
    data = '20'+ str(data)
    return data

# Adiciona a nova coluna com a formatação
dados['Ano'] = dados['Ano'].apply(formata_ano)

# Muda tipo da coluna para concatenação
dados['Ano'] = dados['Ano'].astype(str)

In [266]:
# Extrai mês e cria coluna única
# Define a função para aplicar a cada registro da coluna
def extrair_dia(data):
    return data.strftime('%m')

# Adiciona a nova coluna com os meses extraídos da coluna de data
dados['AnoMes'] = dados['Ano'] + dados['Data da Fatura'].apply(extrair_dia)

# Exclui coluna de ano
dados.drop(columns=['Ano'], inplace = True)

In [86]:
'''
# extrai mes e cria coluna unica
# define a função para aplicar a cada registro da coluna
def extrair_dia(data):
    mes = data.strftime('%m')
    if mes == '01':
        return 'jan'
    elif mes == '02':
        return 'fev'
    elif mes == '03':
        return 'mar'
    elif mes == '04':
        return 'mar'
    elif mes == '05':
        return 'abr'
    elif mes == '06':
        return 'mai'
    elif mes == '07':
        return 'jun'
    elif mes == '08':
        return 'jul'
    elif mes == '08':
        return 'ago'
    elif mes == '09':
        return 'set'
    elif mes == '10':
        return 'out'
    elif mes == '11':
        return 'nov'
    elif mes == '12':
        return 'dez'

# adiciona a nova coluna com os meses extraídos da coluna de data
dados['AnoMes'] = dados['Ano'] + '_' + dados['Data da Fatura'].apply(extrair_dia)

# exclui coluna de ano
dados.drop(columns=['Ano'], inplace = True)'''

In [267]:
dados.head()

,Revendedor,ID Revendedor,Data da Fatura,Regiao,Estado,Cidade,Produto,Preco por Unidade,Unidades Vendidas,Total Venda,Lucro Operacional,Margem de Lucro,Metodo de Venda,AnoMes
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-store,202001
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-store,202001
2,Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,40.0,1000,400000.0,140000.0,0.35,In-store,202001
3,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.0,0.35,In-store,202001
4,Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.0,0.30,In-store,202001


#### Indicador 1 - Faturamento Mensal

In [268]:
dados['Preco por Unidade'].unique()

array([ 50.,  40.,  45.,  60.,  55.,  65.,  70.,  25.,  35.,  30.,  80.,
        75.,  60.,  55.,  45.,  65.,  20.,  30.,  25.,  65.,  75.,  50.,
        85.,  70.,  80., 100.,  75.,  90.,  95.,  50.,  45.,  40.,  50.,
        60.,  25.,  20.,  15.,  35.,  55.,  20.,  10.,  20.,  85.,  95.,
       110.,  90., 105.,  55.,  30.,  40.,  45.,  30.,  15.,  40.,  15.,
        25.,  10.,  35.,  45.,  80.,  90.,  25.,  60.,  35.,  65.,  20.,
        30.,  15.,  25.,  10.,  47.,  36.,  41.,  55.,  46.,  44.,  58.,
        48.,  39.,  43.,  59.,  56.,  49.,  54.,  64.,  53.,  61.,  62.,
        68.,  52.,  66.,  51.,  24.,  34.,  33.,  23.,  32.,  38.,  29.,
        37.,  27.,  42.,  72.,  63.,  69.,  76.,  67.,  57.,  28.,  18.,
        19.,  71.,  78.,  73.,  74.,  83.,  82.,  98.,  77.,  88.,  86.,
        14.,   9.,  15.,  97.,  81.,  79.,  96.,  84.,  89., 103., 101.,
        87.,  10.,  92.,  31.,  26.,  21.,  13.,  22.,  17.,  12.,  16.,
        11.,   7.,   8.])

In [269]:
dados['Preco por Unidade'] = dados['Preco por Unidade'].astype(int)

In [270]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Revendedor         9648 non-null   object        
 1   ID Revendedor      9648 non-null   object        
 2   Data da Fatura     9648 non-null   datetime64[ns]
 3   Regiao             9648 non-null   object        
 4   Estado             9648 non-null   object        
 5   Cidade             9648 non-null   object        
 6   Produto            9648 non-null   object        
 7   Preco por Unidade  9648 non-null   int32         
 8   Unidades Vendidas  9648 non-null   int32         
 9   Total Venda        9648 non-null   float64       
 10  Lucro Operacional  9648 non-null   float64       
 11  Margem de Lucro    9648 non-null   float64       
 12  Metodo de Venda    9648 non-null   object        
 13  AnoMes             9648 non-null   object        
dtypes: datet

In [272]:
dados['Faturamento'] = dados['Preco por Unidade'] * dados['Unidades Vendidas']

In [273]:
# Agrupa o faturamento por mês/ano
df_faturamento = dados.groupby(['AnoMes']).agg({'Faturamento':sum}).reset_index()

In [274]:
df_faturamento.head()

,AnoMes,Faturamento
0,202001,2311271
1,202002,2140813
2,202003,2473102
3,202004,3192731
4,202005,2164764


In [275]:
# Visualização

# Definição dos dados no plot
plot_data = [go.Scatter(x = df_faturamento['AnoMes'],
                        y = df_faturamento['Faturamento'],)]

plot_layout = go.Layout(xaxis = {'type': 'category'},
                        title = 'Faturamento Mensal')

# Plotagem
fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

#### Indicador 2 - Taxa Percentual de Crescimento Mensal

Taxa Percentual de Crescimento Mensal = (Faturamento Mensal / Faturamento Mensal Anterior) * 100

In [276]:
# Variação percentual mensal
df_faturamento['CrescimentoMensal'] = df_faturamento['Faturamento'].pct_change() * 100

In [277]:
df_faturamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AnoMes             24 non-null     object 
 1   Faturamento        24 non-null     int32  
 2   CrescimentoMensal  23 non-null     float64
dtypes: float64(1), int32(1), object(1)
memory usage: 608.0+ bytes


In [279]:
df_faturamento['AnoMes'] = df_faturamento['AnoMes'].astype(int)

In [280]:
df_faturamento

,AnoMes,Faturamento,CrescimentoMensal
0,202001,2311271,NaN
1,202002,2140813,-7.375076
2,202003,2473102,15.521627
3,202004,3192731,29.098234
4,202005,2164764,-32.197106
5,202006,1083069,-49.968264
6,202007,2182388,101.500366
7,202008,2641630,21.043096
8,202009,2367686,-10.370264
9,202010,1428269,-39.676587


In [281]:
# Visualização de 2020

# Filtrar apenas ano 2020
plot_data = [go.Scatter(x = df_faturamento.query('AnoMes < 202101')['AnoMes'],
                        y = df_faturamento.query('AnoMes < 202101')['CrescimentoMensal'],)]

plot_layout = go.Layout(xaxis = {'type': 'category'},
                        title = 'Taxa Percentual de Crescimento Mensal (em %)')

# Plotagem
fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

In [145]:
# Visualização de 2021

# Filtrar apenas ano 2021
plot_data = [go.Scatter(x = df_faturamento.query('AnoMes > 202012')['AnoMes'],
                        y = df_faturamento.query('AnoMes > 202012')['CrescimentoMensal'],)]

plot_layout = go.Layout(xaxis = {'type': 'category'},
                        title = 'Taxa Percentual de Crescimento Mensal (em %)')

# Plotagem
fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

In [282]:
# Visualização completa

plot_data = [go.Scatter(x = df_faturamento['AnoMes'],
                        y = df_faturamento['CrescimentoMensal']),]

plot_layout = go.Layout(xaxis = {'type': 'category'},
                        title = 'Taxa Percentual de Crescimento Mensal (em %)')

# Plotagem
fig = go.Figure(data = plot_data, layout = plot_layout)
#fig.update_xaxes(categoryorder='array', categoryarray= ordem_eixox)
pyoff.iplot(fig)

### Indicador 3 - Revendedores Ativos Por Mês

In [283]:
df_revendedores = dados.groupby('AnoMes')['ID Revendedor'].nunique().reset_index()

In [284]:
df_revendedores.head()

,AnoMes,ID Revendedor
0,202001,2
1,202002,2
2,202003,2
3,202004,2
4,202005,2


In [285]:
# Visualização

plot_data = [go.Bar(x = df_revendedores['AnoMes'],
                   y = df_revendedores['ID Revendedor'])]

plot_layout = go.Layout( xaxis = {'type':'category'},
                        title = 'Revendedores Ativos por Mês')

fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

### Indicador 4 - Total de Produtos Vendidos Por Mês

In [286]:
df_produtos = dados.groupby(['AnoMes','Produto']).agg({'Unidades Vendidas': sum}). reset_index()

In [287]:
# Acrescentar palavras na coluna AnoMes, pois o plotly não estava detectando como string com o plotly.express
# Define a função para aplicar a cada registro da coluna
def formata_ano(data):
    data = 'Mes_'+ str(data)
    return data

# Adiciona a nova coluna com a formatação
df_produtos['AnoMes'] = df_produtos['AnoMes'].apply(formata_ano)

In [289]:
# Visualização

fig = px.line(df_produtos, x = 'AnoMes', y = df_produtos[df_produtos.columns[2]], color = 'Produto',
               title = 'Unidades Vendidas por Produto')
fig.show()

### Indicador 5 - Faturamento Mensal por Revendedor

In [290]:
df_faturamento_revendedores = dados.groupby(['AnoMes','Revendedor']).agg({'Faturamento':sum}).reset_index()

In [291]:
df_faturamento_revendedores.head()

,AnoMes,Revendedor,Faturamento
0,202001,Foot Locker,1399680
1,202001,West Gear,911591
2,202002,Foot Locker,714912
3,202002,West Gear,1425901
4,202003,Foot Locker,1470219


In [293]:
# Acrescentar palavras na coluna AnoMes, pois o plotly não estava detectando como string com o plotly.express
# Define a função para aplicar a cada registro da coluna
def formata_ano(data):
    data = 'Mes_'+ str(data)
    return data

# Adiciona a nova coluna com a formatação
df_faturamento_revendedores['AnoMes'] = df_faturamento_revendedores['AnoMes'].apply(formata_ano)

In [ ]:
'''fig = px.line(df_faturamento_revendedores, 
              x = 'AnoMes', 
              y = 'Faturamento', 
              color = 'Revendedor',
              title = 'Faturamento Mensal por Revendedor')
fig.show()'''

In [197]:
# Visualização

fig = px.bar(df_faturamento_revendedores, 
             x = 'AnoMes', 
             y = 'Faturamento',
             #barmode = 'group', 
             color = 'Revendedor')
fig.show()


### Indicador 6 - Faturamento Médio Mensal

In [296]:
# Faturamento médio mensal
df_faturamento_medio = dados.groupby('AnoMes')['Faturamento'].mean().reset_index()

In [297]:
df_faturamento_medio.head()

,AnoMes,Faturamento
0,202001,15408.473333
1,202002,18779.061404
2,202003,18878.641221
3,202004,24559.469231
4,202005,22786.989474


In [299]:
# Visualização

plot_data = [go.Bar(x = df_faturamento_medio['AnoMes'],
                    y = df_faturamento_medio['Faturamento'],
                    marker = dict(color = 'green')),]

plot_layout = go.Layout( xaxis = {'type':'category'},
                        title = 'Fatuamento Médio Mensal')

fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

In [300]:
# Faturamento total mensal
df_faturamento_total = dados.groupby('AnoMes')['Faturamento'].sum().reset_index()

In [301]:
# Visualização

plot_data = [go.Bar(x = df_faturamento_total['AnoMes'],
                    y = df_faturamento_total['Faturamento'],
                    marker = dict(color = 'green'))]

plot_layout = go.Layout(xaxis = {'type':'category'},
                        title = 'Fatuamento Total Mensal')

fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

A média mensal de faturamento de 2020 é maior que em 2021, porém o valor total faturado mensal em 2020 é menor que em 2021. Isso significa houve menos vendas em 2020, no entanto essas vendas teve um valor alto de cada produto, ao passo que em 2021 houve mais vendas de produtos com menor valor.